# Load pyrouge

In [108]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path 'pyrouge/tools/ROUGE-1.5.5'

Requirement already up-to-date: pyrouge in /usr/local/lib/python3.7/dist-packages (0.1.3)
  Using cached https://github.com/bheinzerling/pyrouge/archive/master.zip
  Created wheel for pyrouge: filename=pyrouge-0.1.3-cp37-none-any.whl size=191921 sha256=030d3f8b2c48bed8b41d2e251504f646a4000a594176f2ef51d9943ccfe8570f
  Stored in directory: /tmp/pip-ephem-wheel-cache-0ey6mqdv/wheels/70/02/b4/a23b5feb5980a5eb940441cb04ec1e17d5f18344138efbecf8
Successfully built pyrouge
Name: pyrouge
Version: 0.1.3
Summary: A Python wrapper for the ROUGE summarization evaluation package.
Home-page: https://github.com/noutenki/pyrouge
Author: Benjamin Heinzerling, Anders Johannsen
Author-email: benjamin.heinzerling@h-its.org
License: LICENSE.txt
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 
Cloning into 'pyrouge'...
remote: Enumerating objects: 393, done.
remote: Total 393 (delta 0), reused 0 (delta 0), pack-reused 393
Receiving objects: 100% (393/393), 298.74 KiB | 1.55 MiB/s, d

In [109]:
!pip install transformers
!pip install tensorboardX
!pip install easydict

In [110]:
%cd /content
!git clone https://github.com/HaloKim/KorBertSum.git
%cd /content/KorBertSum

/content
fatal: destination path 'KorBertSum' already exists and is not an empty directory.
/content/KorBertSum


# Code

In [111]:
import os

os.chdir('/content/KorBertSum/src')

In [112]:
import torch
import numpy as np
from models import data_loader, model_builder
from models.model_builder import Summarizer
from others.logging import logger, init_logger
from models.data_loader import load_dataset
from transformers import BertConfig
from tensorboardX import SummaryWriter
from models.reporter import ReportMgr
from models.stats import Statistics
import easydict
from transformers import BertTokenizer

# Preprocessing

In [113]:
def _tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

def build_trainer(args, device_id, model,
                  optim):
    """
    Simplify `Trainer` creation based on user `opt`s*
    Args:
        opt (:obj:`Namespace`): user options (usually from argument parsing)
        model (:obj:`onmt.models.NMTModel`): the model to train
        fields (dict): dict of fields
        optim (:obj:`onmt.utils.Optimizer`): optimizer used during training
        data_type (str): string describing the type of data
            e.g. "text", "img", "audio"
        model_saver(:obj:`onmt.models.ModelSaverBase`): the utility object
            used to save the model
    """
    device = "cpu" if args.visible_gpus == '-1' else "cuda"


    grad_accum_count = args.accum_count
    n_gpu = args.world_size

    if device_id >= 0:
        gpu_rank = int(args.gpu_ranks[device_id])
    else:
        gpu_rank = 0
        n_gpu = 0

    print('gpu_rank %d' % gpu_rank)

    tensorboard_log_dir = args.model_path

    writer = SummaryWriter(tensorboard_log_dir, comment="Unmt")

    report_manager = ReportMgr(args.report_every, start_time=-1, tensorboard_writer=writer)

    trainer = Trainer(args, model, optim, grad_accum_count, n_gpu, gpu_rank, report_manager)

    # print(tr)
    if (model):
        n_params = _tally_parameters(model)
        logger.info('* number of parameters: %d' % n_params)

    return trainer


class Trainer(object):
    """
    Class that controls the training process.

    Args:
            model(:py:class:`onmt.models.model.NMTModel`): translation model
                to train
            train_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            valid_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            optim(:obj:`onmt.utils.optimizers.Optimizer`):
               the optimizer responsible for update
            trunc_size(int): length of truncated back propagation through time
            shard_size(int): compute loss in shards of this size for efficiency
            data_type(string): type of the source input: [text|img|audio]
            norm_method(string): normalization methods: [sents|tokens]
            grad_accum_count(int): accumulate gradients this many times.
            report_manager(:obj:`onmt.utils.ReportMgrBase`):
                the object that creates reports, or None
            model_saver(:obj:`onmt.models.ModelSaverBase`): the saver is
                used to save a checkpoint.
                Thus nothing will be saved if this parameter is None
    """

    def __init__(self,  args, model,  optim,
                  grad_accum_count=1, n_gpu=1, gpu_rank=1,
                  report_manager=None):
        # Basic attributes.
        self.args = args
        self.save_checkpoint_steps = args.save_checkpoint_steps
        self.model = model
        self.optim = optim
        self.grad_accum_count = grad_accum_count
        self.n_gpu = n_gpu
        self.gpu_rank = gpu_rank
        self.report_manager = report_manager

        self.loss = torch.nn.BCELoss(reduction='none')
        assert grad_accum_count > 0
        # Set model in training mode.
        if (model):
            self.model.train()

    def summ(self, test_iter, step, cal_lead=False, cal_oracle=False):
      """ Validate model.
          valid_iter: validate data iterator
      Returns:
          :obj:`nmt.Statistics`: validation loss statistics
      """
      # Set model in validating mode.
      def _get_ngrams(n, text):
          ngram_set = set()
          text_length = len(text)
          max_index_ngram_start = text_length - n
          for i in range(max_index_ngram_start + 1):
              ngram_set.add(tuple(text[i:i + n]))
          return ngram_set

      def _block_tri(c, p):
          tri_c = _get_ngrams(3, c.split())
          for s in p:
              tri_s = _get_ngrams(3, s.split())
              if len(tri_c.intersection(tri_s))>0:
                  return True
          return False

      if (not cal_lead and not cal_oracle):
          self.model.eval()
      stats = Statistics()

      with torch.no_grad():
          for batch in test_iter:
              src = batch.src
              labels = batch.labels
              segs = batch.segs
              clss = batch.clss
              mask = batch.mask
              mask_cls = batch.mask_cls

              if (cal_lead):
                  selected_ids = [list(range(batch.clss.size(1)))] * batch.batch_size
              elif (cal_oracle):
                  selected_ids = [[j for j in range(batch.clss.size(1)) if labels[i][j] == 1] for i in
                                  range(batch.batch_size)]
              else:
                  sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)
                  sent_scores = sent_scores + mask.float()
                  sent_scores = sent_scores.cpu().data.numpy()
                  selected_ids = np.argsort(-sent_scores, 1)
      return selected_ids



    def _gradient_accumulation(self, true_batchs, normalization, total_stats,
                               report_stats):
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            if self.grad_accum_count == 1:
                self.model.zero_grad()

            src = batch.src
            labels = batch.labels
            segs = batch.segs
            clss = batch.clss
            mask = batch.mask
            mask_cls = batch.mask_cls

            sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

            loss = self.loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()
            (loss/loss.numel()).backward()
            # loss.div(float(normalization)).backward()

            batch_stats = Statistics(float(loss.cpu().data.numpy()), normalization)


            total_stats.update(batch_stats)
            report_stats.update(batch_stats)

            # 4. Update the parameters and statistics.
            if self.grad_accum_count == 1:
                # Multi GPU gradient gather
                if self.n_gpu > 1:
                    grads = [p.grad.data for p in self.model.parameters()
                             if p.requires_grad
                             and p.grad is not None]
                    distributed.all_reduce_and_rescale_tensors(
                        grads, float(1))
                self.optim.step()

        # in case of multi step gradient accumulation,
        # update only after accum batches
        if self.grad_accum_count > 1:
            if self.n_gpu > 1:
                grads = [p.grad.data for p in self.model.parameters()
                         if p.requires_grad
                         and p.grad is not None]
                distributed.all_reduce_and_rescale_tensors(
                    grads, float(1))
            self.optim.step()

    def _save(self, step):
        real_model = self.model
        # real_generator = (self.generator.module
        #                   if isinstance(self.generator, torch.nn.DataParallel)
        #                   else self.generator)

        model_state_dict = real_model.state_dict()
        # generator_state_dict = real_generator.state_dict()
        checkpoint = {
            'model': model_state_dict,
            # 'generator': generator_state_dict,
            'opt': self.args,
            'optim': self.optim,
        }
        checkpoint_path = os.path.join(self.args.model_path, 'model_step_%d.pt' % step)
        logger.info("Saving checkpoint %s" % checkpoint_path)
        # checkpoint_path = '%s_step_%d.pt' % (FLAGS.model_path, step)
        if (not os.path.exists(checkpoint_path)):
            torch.save(checkpoint, checkpoint_path)
            return checkpoint, checkpoint_path

    def _start_report_manager(self, start_time=None):
        """
        Simple function to start report manager (if any)
        """
        if self.report_manager is not None:
            if start_time is None:
                self.report_manager.start()
            else:
                self.report_manager.start_time = start_time

    def _maybe_gather_stats(self, stat):
        """
        Gather statistics in multi-processes cases

        Args:
            stat(:obj:onmt.utils.Statistics): a Statistics object to gather
                or None (it returns None in this case)

        Returns:
            stat: the updated (or unchanged) stat object
        """
        if stat is not None and self.n_gpu > 1:
            return Statistics.all_gather_stats(stat)
        return stat

    def _maybe_report_training(self, step, num_steps, learning_rate,
                               report_stats):
        """
        Simple function to report training stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_training` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_training(
                step, num_steps, learning_rate, report_stats,
                multigpu=self.n_gpu > 1)

    def _report_step(self, learning_rate, step, train_stats=None,
                     valid_stats=None):
        """
        Simple function to report stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_step` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_step(
                learning_rate, step, train_stats=train_stats,
                valid_stats=valid_stats)

    def _maybe_save(self, step):
        """
        Save the model if a model saver is set
        """
        if self.model_saver is not None:
            self.model_saver.maybe_save(step)
            
def _lazy_dataset_loader(pt_file):
  yield  pt_file

# Params

In [114]:
args = easydict.EasyDict({
    "encoder":'classifier',
    "mode":'test',
    "bert_data_path":'/content/drive/MyDrive/Colab_Notebooks/Study/BertSum-master/bert_data/korean',
    "model_path":'./models/bert_classifier',
    "result_path":'./results',
    "temp_dir":'./temp',
    "batch_size":1000,
    "use_interval":True,
    "hidden_size":128,
    "ff_size":512,
    "heads":4,
    "inter_layers":2,
    "rnn_size":512,
    "param_init":0,
    "param_init_glorot":True,
    "dropout":0.1,
    "optim":'adam',
    "lr":2e-3,
    "report_every":1,
    "save_checkpoint_steps":5,
    "block_trigram":True,
    "recall_eval":False,
    
    "accum_count":1,
    "world_size":1,
    "visible_gpus":'-1',
    "gpu_ranks":'0',
    "log_file":'/content/drive/MyDrive/Colab_Notebooks/Study/BertSum-master/logs/log.log',
    "test_from":'/content/drive/MyDrive/Colab_Notebooks/Study/BertSum-master/models/bert_classifier/model_step_10000.pt'
})
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']


# Test code

In [115]:
ef test(args, input_list, device_id, pt, step):
  init_logger(args.log_file)
  device = "cpu" if args.visible_gpus == '-1' else "cuda"
  device_id = 0 if device == "cuda" else -1

  cp = args.test_from
  try:
    step = int(cp.split('.')[-2].split('_')[-1])
  except:
    step = 0

  device = "cpu" if args.visible_gpus == '-1' else "cuda"
  if (pt != ''):
      test_from = pt
  else:
      test_from = args.test_from
  logger.info('Loading checkpoint from %s' % test_from)
  checkpoint = torch.load(test_from, map_location=lambda storage, loc: storage)
  opt = vars(checkpoint['opt'])
  for k in opt.keys():
      if (k in model_flags):
        setattr(args, k, opt[k])

  config = BertConfig.from_pretrained('bert-base-multilingual-cased')
  model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
  model.load_cp(checkpoint)
  model.eval()

  test_iter = data_loader.Dataloader(args, _lazy_dataset_loader(input_list),
                                args.batch_size, device,
                                shuffle=False, is_test=True)
  trainer = build_trainer(args, device_id, model, None)
  result = trainer.summ(test_iter,step)
  return result, input_list

args.gpu_ranks = [int(i) for i in args.gpu_ranks.split(',')]
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

In [120]:
def txt2input(text):
  data = list(filter(None, text.split('\n')))
  bertdata = BertData()
  txt_data = bertdata.preprocess(data)
  data_dict = {"src":txt_data[0],
               "labels":[0,1,2],
               "segs":txt_data[2],
               "clss":txt_data[3],
               "src_txt":txt_data[4],
               "tgt_txt":None}
  input_data = []
  input_data.append(data_dict)
  return input_data

[2021-06-09 05:05:43,764 INFO] Loading checkpoint from /content/drive/MyDrive/Colab_Notebooks/Study/BertSum-master/models/bert_classifier/model_step_10000.pt
[2021-06-09 05:05:49,333 INFO] * number of parameters: 177854209


gpu_rank 0


array([0, 2, 6])

In [119]:
text = '''
[이데일리 박지혜 기자] 조상호 더불어민주당 전 부대변인이 결국 “천안함 함장이 부하들을 수장시켰다”는 발언에 대해 사과했다.

조 전 부대변인은 9일 페이스북을 통해 “제 주변 분들의 애정 어린 권고가 있었다”고 운을 뗐다.

그러면서 “제 표현 중 혹여 순국한 46 용사의 유가족, 특히 아직도 시신조차 거두지 못한 6인의 유가족과 피해 장병들에게 고통스런 기억을 떠올리게 한 부분이 있다는 지적, 깊게 받아드린다”며 “상처로 떠올리신 유가족과 피해 장병께는 진심으로 사죄드린다”고 전했다.

이어 “다시 한 번 46 용사의 명복을 빈다”고 덧붙였다.


조상호 더불어민주당 전 부대변인 (사진=채널A 방송 캡처)
앞서 조 전 부대변인은 지난 7일 오후 종합편성채널 채널A ‘뉴스톱10’에서 “최원일 전 함장이라는 예비역 대령, 그분도 승진했다. 그런데 그분은 그(처우 관련) 말을 할 자격이 없다”며 “최 전 함장이 그때 당시 생때같은 자기 부하들을 다 수장시켜 놓고 이후에 제대로 된 책임이 없었다”고 말했다.

당시 방송하던 진행자와 다른 출연자들이 최 함장이 수장시킨 건 아니라며 발언을 제지했지만, 조 전 부대변인은 주장을 굽히지 않았다. 이후 자신의 페이스북에도 “도대체 뭐가 막말이냐”는 글을 올렸다.

그는 또 “작전에 실패한 군인은 몰라도 경계에 실패한 군인은 용서할 수 없다는 군사 격언이 있다”며 “심지어 당시는 한미연합훈련 중이었다. 하지만 함장 지휘관이 폭침으로 침몰 되는데도 뭐에 당했는지도 알지 못 했다”고 했다.

그러면서 “결국 46명의 젊은 목숨을 잃었다. 근데 함장이 책임이 없나”고 반문했다.

한편, 송영길 민주당 대표도 이날 조 전 부대변인의 발언에 항의한 최 전 함장과 유가족에게 “죄송하다”고 사과했다.

최 전 함장과 천안함 유가족은 이날 여의도 국회를 찾아 송 대표를 면담하고 공식 사과를 요구했다.

송 대표는 이 자리에서 “당 대표로서 죄송하다”며 “조 전 부대변인의 잘못된 언어 사용에 대해서 유감을 표명한다”고 밝힌 것으로 전해졌다.

고용진 수석대변인은 이날 면담 후 “조 전 대변인은 아무 당직 없이 당적만 보유한 분이며, 그분의 의견은 당과는 전혀 관련없는 의견”이라고 설명했다.

그는 “함장이 수장시켰다는 식으로 발언한 것은 사과해야 한다고 (조 전 대변인에게) 요구하고 있다”며 “김병주 의원도 참석했는데, 국방위에서 천안함 폭침이 분명히 북한 소행이라는 점을 말할 것”이라고 전했다.

박지혜 (noname@edaily.co.kr)
'''

In [ ]:
input_data = txt2input(text)
sum_list = test(args, input_data, -1, '', None)
sum_list[0][:3]

# Result

In [131]:
[list(filter(None, text.split('\n')))[i] for i in sum_list[0][:3]]

['[이데일리 박지혜 기자] 조상호 더불어민주당 전 부대변인이 결국 “천안함 함장이 부하들을 수장시켰다”는 발언에 대해 사과했다.',
 '그러면서 “제 표현 중 혹여 순국한 46 용사의 유가족, 특히 아직도 시신조차 거두지 못한 6인의 유가족과 피해 장병들에게 고통스런 기억을 떠올리게 한 부분이 있다는 지적, 깊게 받아드린다”며 “상처로 떠올리신 유가족과 피해 장병께는 진심으로 사죄드린다”고 전했다.',
 '당시 방송하던 진행자와 다른 출연자들이 최 함장이 수장시킨 건 아니라며 발언을 제지했지만, 조 전 부대변인은 주장을 굽히지 않았다. 이후 자신의 페이스북에도 “도대체 뭐가 막말이냐”는 글을 올렸다.']